In [1]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
from tqdm import tqdm

In [3]:
df_id = pd.read_csv('product_id.csv')
p_ids = df_id.id.to_list()
result = []

In [9]:
headers = {
'accept':'application/json, text/plain, */*',
'accept-encoding':'gzip, deflate, br, zstd',
'accept-language':'en-US,en;q=0.9',
'priority':'u=1, i',
'referer':'https://tiki.vn/may-doc-sach-all-new-kindle-paperwhite-5-11th-gen-hang-nhap-khau-p125182567.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.204054_Y.1786374_Z.3548453_CN.PPW5--Chuan-nhat&itm_medium=CPC&itm_source=tiki-ads&spid=125182569',
'sec-ch-ua':'"Chromium";v="128", "Not;A=Brand";v="24", "Microsoft Edge";v="128"',
'sec-ch-ua-mobile':'?0',
'sec-ch-ua-platform':'"Windows"',
'sec-fetch-dest':'empty',
'sec-fetch-mode':'cors',
'sec-fetch-site':'same-origin',
'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
'x-guest-token':'y0U43MiCJhumQfcxjdO6YrS5LIBgKwp8'
}

In [5]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['url']= json.get('url_key')
    d['name']= json.get('name')
    d['price'] = json.get('price')
    if json.get('images') is not None:
        d['images_base_url'] = json.get('images')[0].get('base_url')
    else:
        d['images_base_url'] = None
    d['description']= BeautifulSoup(json.get('description'),'html.parser').get_text()
    return d

In [68]:
def save_state(index):
    with open('state.json', 'w') as f:
        json.dump({'index': index}, f)

def load_state():
    try:
        with open('state.json', 'r') as f:
            return json.load(f)['index']
    except FileNotFoundError:
        return 0  # Bắt đầu từ 0 nếu không tìm thấy file
start_index = load_state()
for i in tqdm(range(start_index,len(p_ids))):
    response = requests.get('https://api.tiki.vn/product-detail/api/v1/products/{}'.format(p_ids[i]),headers = headers)
    if response.status_code == 200:
        print('Crawl data {} successfull'.format(p_ids[i]))
        result.append(parser_product(response.json()))
        save_state(i+1)
    elif response.status_code == 404:
        pass
    else:
        print(f"Error fetching data for item {i}: {response.status_code}")
        break  # Dừng tại đây nếu có lỗi

0it [00:00, ?it/s]


In [ ]:
df_product = pd.DataFrame(result)
df_product.to_csv('crawled_data_ncds.csv', index=False)

In [64]:
len(result)

199264

In [60]:
df_product = pd.DataFrame(result)

In [72]:
df_product.to_csv('crawled_data_ncds.csv', index=False)

In [70]:
df_product.tail()

,id,url,name,price,images_base_url,description
199259,145236648,ca-vat-nam-su-kien-hoi-nghi-du-tiec-cong-so-8c...,"CÀ VẠT NAM SỰ KIỆN, HỘI NGHỊ, DỰ TIỆC, CÔNG SỞ...",167000,https://salt.tikicdn.com/ts/product/2c/bd/65/3...,\n\n\n\n CÀ VẠT NAM THỜI TRANG FITTOP\n-------...
199260,26470905,khan-len-nam-nu-phong-cach-han-quoc-dai-1-8m-r...,Khăn len nam nữ phong cách hàn quốc (dài 1.8m ...,109000,https://salt.tikicdn.com/ts/product/c0/0f/d5/0...,THÔNG TIN SẢN PHẨM\nChất liệu :\nLen đan ấm áp...
199261,31288300,khan-len-quang-co-nam-lich-lam-sang-trong-sk10...,Khăn len quảng cổ nam lịch lãm sang trọng SK108,109000,https://salt.tikicdn.com/ts/product/2e/1a/04/0...,"Khăn len nam\nKhông chỉ riêng nữ giới, các đấn..."
199262,194026976,khan-tay-khan-choang-tam-doc-dao-thu-hut-moi-a...,Khăn tay khăn choàng tắm độc đáo thu hút mọi á...,39200,https://salt.tikicdn.com/ts/product/a8/aa/5c/b...,ĐẶC ĐIỂM NỔI BẬT\nKhăn tay khăn choàng tắm độc...
199263,6896803,khan-bit-mat-chong-nang-di-phuot-cs-p6896803,Khăn Bit Mặt Chống Nắng Đi Phượt CS,35000,https://salt.tikicdn.com/ts/product/2b/0f/a4/2...,Khăn Trùm Full Đầu Ninja - Lựa Chọn Của Dân Ph...


In [27]:
result[-1]

{'id': 205514048,
 'url': 'phu-kien-luoi-dao-cao-rau-3-luoi-thay-the-cho-may-cao-rau-mini-3-luoi-cong-nghe-nhat-ban-p205514048',
 'name': 'Phụ Kiện - Lưỡi Dao Cạo Râu 3 Lưỡi Thay Thế Cho Máy Cạo Râu MiNi 3 Lưỡi Công Nghệ Nhật Bản',
 'price': 35000,
 'images_base_url': 'https://salt.tikicdn.com/ts/product/e8/a0/d0/47d0131555d2aa339c3b42521ea5cd05.jpg',
 'description': 'Lưỡi Cạo Thay Thế Cho Máy Cạo Râu Mini Siêu Nhỏ - Mẫu Lưỡi Cao Thiết Kế Theo Công Nghệ Nhật Bản\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\n️ Dao cạo râu tốt, an toàn, chất lượng\n– Vật dụng cần thiết của cánh mày râu\n️ Đối với cánh mày râu, dao cạo râu được xem là một dụng cụ không thể thiếu để chăm sóc bản thân. Nó sẽ giúp dọn sạch râu, lông mang đến cho bạn một gương mặt điển trai, sạch sẽ, đầy sức hút. Thế nhưng, mua dao cạo ở đâu để đảm bảo chất lượng thì vẫn là băn khoăn của nhiều người. Giờ đây, công việc mua sắm này đã trở nên dễ dàng 

In [33]:
with open('state.json', 'w') as f:
    json.dump({'index': 38718}, f)

In [43]:
df_product.to_csv('crawled_data_ncds.csv', index=False)

In [66]:
load_state()

200000

In [84]:
df_product.head()

,id,url,name,price,images_base_url,description
0,1391347,tranh-xep-hinh-tia-sang-guong-thu-tia-sang-203...,Tranh xếp Hình Tia Sáng Gương Thu Tia Sáng (2...,209950,https://salt.tikicdn.com/ts/product/f4/6e/ff/1...,Bộ Xếp Hình Gương Thu Tia Sáng (2035 Mảnh Ghép...
1,74897599,bo-do-choi-lap-ghep-xe-xuc-kavy-nhieu-mau-sac-...,Bộ đồ chơi lắp ghép xe xúc KAVY nhiều màu sắc ...,87000,https://salt.tikicdn.com/ts/product/75/3f/28/a...,BỘ ĐỒ CHƠI LẮP RÁP XE XÚC CHO BÉƯU ĐIỂM :+ Chấ...
2,154155413,bo-do-choi-tau-hoa-do-choi-xe-lua-co-duong-ray...,"Bộ đồ chơi tàu hỏa, Đồ chơi xe lửa có đường ra...",59000,https://salt.tikicdn.com/ts/product/a6/f2/d0/b...,"Bộ đồ chơi tàu hỏa, Đồ chơi xe lửa có đường ra..."
3,253117062,mo-hinh-lego-minifigures-lap-rap-nhan-vat-hoat...,Mô Hình Minifigures Lắp Ráp Nhân Vật Hoạt Hình...,18000,https://salt.tikicdn.com/ts/product/1c/19/03/b...,• Chiều cao mỗi nhân vật từ 4 - 5cm• Sản phẩm ...
4,130978358,bo-do-choi-heo-peppa-leo-thang-co-nhac-va-den-...,Bộ đồ chơi Heo Peppa leo thang - Có nhạc và đè...,90000,https://salt.tikicdn.com/ts/product/cf/a5/7b/7...,Thông tin sản phẩm:- Tên sản phẩm: Đồ chơi heo...


In [82]:
df_product['description']=df_product['description'].str.replace('\n','')

In [86]:
df_product.to_csv('crawled_data_ncds.csv', index=False)